### Import Data & Convert Dataframe to Dataset Dictionary

In [1]:
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split

# Loading the dataset
data = pd.read_csv('WELFake_Dataset.csv')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [3]:
data = data.drop(['Unnamed: 0', 'title'], axis=1)

In [4]:
data.isna().sum()

text     39
label     0
dtype: int64

In [5]:
data = data.dropna()
data.isna().sum()

text     0
label    0
dtype: int64

In [6]:
data

,text,label
0,No comment is expected from Barack Obama Membe...,1
1,Did they post their votes for Hillary already?,1
2,"Now, most of the demonstrators gathered last ...",1
3,A dozen politically active pastors came here f...,0
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...
72129,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,"You know, because in fantasyland Republicans n...",1
72131,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


### Split Train-Test-Validation Data

In [7]:
# Split 1
train, test = train_test_split(data, test_size=0.3, shuffle = True, random_state = 8)

In [8]:
# Split 2
train, val = train_test_split(train, test_size=0.3, shuffle = True, random_state = 8)

In [9]:
# Assign the Train/Test/Val 
train_data = datasets.DatasetDict({'train': datasets.Dataset.from_pandas(train, preserve_index=False),
                                   'test':datasets.Dataset.from_pandas(test,  preserve_index=False), 
                                   'validation':datasets.Dataset.from_pandas(val,  preserve_index=False)})

In [10]:
train_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 35326
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21629
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 15140
    })
})

### Tokenize  Data

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
def preprocess_function(row):
    token = tokenizer(row["text"], truncation=True)
    row["input_ids"] = token["input_ids"]
    row["attention_mask"] = token["attention_mask"]
    return row

In [13]:
train_data = train_data.map(preprocess_function)

  0%|          | 0/35326 [00:00<?, ?ex/s]

  0%|          | 0/21629 [00:00<?, ?ex/s]

  0%|          | 0/15140 [00:00<?, ?ex/s]

In [14]:
train_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 35326
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 21629
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 15140
    })
})

### Transformers Model

In [16]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [35]:
tf_train_set = train_data['train'].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = train_data["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [36]:
from transformers import TFBertModel
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [37]:
import tensorflow as tf
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)

In [38]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
intermediate_layer (Dense)      (None, 512)          393728      bert[0][1]            

In [39]:
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss=tf.keras.losses.SparseCategoricalCrossentropy()
metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [40]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

In [43]:
history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=2)

Epoch 1/2


ValueError: in user code:

    C:\Users\brend\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\brend\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\brend\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\brend\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\brend\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\brend\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 